<i> Copyright &copy; 2025 Johns Hopkins University.  Not for distribution online or by any other means.</i>

John Doe, Programming Assignment 4, Algorithms 605.621  **PUT YOUR NAME ON THE ASSIGNMENT**

***Statement of Academic Integrity:*** Put your statement of academic integrity here.  Failure to do so will result in deduction of points.

# Overview

In this assignment, we will continue to explore the Maryland Lighthouse Challenge problem.  In PA 2, you implemented a brute-force solution to the problem.  This is guaranteed to get the optimal solution, but it was extremely slow to run.  In PA 3, you implemented a technique called memoization, which traded memory for computation to speed up the results.  However, on very large data sets, this is still impractical, since the memory required for storing the intermediate results may also be excessively large.

In this assignment, we're going to give up on the requirement of finding the best possible path, and instead see if we can get an approximate solution that will be "good enough".  We will want to try to come to an understanding of both how much computation we will save, and how much error we will incur as a result.

The approximation technique we will use is a simple, greedy algorithm:  the algorithm will start at a (randomly-chosen) start light, and then move to the nearest neighbor left in the list.  This will continue until all lighthouses have been visited.  The exact path chosen, and thus the error introduced by the greedy algorithm, will vary depending on the start light and on the specific geometry of the lighthouse layout.  So, a point estimate of the performance will likely be misleading.  To handle this issue, you will be implementing a *Monte Carlo experiment*-- you will run the data collection multiple times, to collect an ensemble of error values, and then draw conclusions from a statistical analysis of the error data.

For this problem, you should use the following assumptions:
1. The contest requires a visit of all lighthouses; the winner is the first team to visit all lights (the contestants do not need to return to the start point).
1. Teams may start at any lighthouse, and finish at any other lighthouse. 
1. Travel time from A to B is the same as the time from B to A. (Equivalently, the paths A->B->C and C->B->A are equal, and your algorithm is correct if it provides either one.  You do not need to provide both.)
1. Several utility functions are provided for your use-- you may modify these as you see fit, or write your own.  Your fastest_tour_bf() program from PA 2 will be a utility function for you to use in this assignment, to calculate the error introduced by our approximation algorithm, and to compare the reduction in workload we achieve.


# Restrictions

1. You should only use Python inbuilt data structures for this assignment (e.g. lists, dictionaries, tuples, etc.)  *Hint:* Pay close attention to the rules on whether a Python data type is *mutable* (changeable) or *immutable* (not)-- remember that only *immutable* data items (like tuples) should be used as keys to dictionaries.
1. You may not use Python libraries for combinatorics, sorting, permutations, combinations, or similar tools.  You need to build all logical structures that traverse the data set yourself, so that you can accurately measure the workload imposed by your algorithmic choices.
1. Do not use Python's sort() function (or any other library function for sorting or ordering lists).  You will underestimate the workload imposed by your algorithm if you use the Python libraries.  An implementation of mergesort() is given to you in the utility programs block.  If you need to sort data, use this function and add an appropriate step-count variable to the code to account for the induced workload.
1. When in doubt, **ASK FIRST**
1. This is not a collaborative problem-- all the work in this notebook should be your individual effort
1. Don't change the signatures of the functions or the kickoff code, as this is used by the grader for automated correctness checking.  You will get points off if you modify the input and output blocks (noted with "DO NOT MODIFY THIS BLOCK") or if your code doesn't execute without modification.  
1. If you didn't get your code working correctly in PA 2, contact the instructor for assistance.


# PART 1:  IMPLEMENTATION

## 1a. Pseudocode (10 pts)

Given a start light and a list of lighthouses, use the nearest neighbor approximation to find the approximate best path
	
<!--- This is a Markdown comment. -->
<!--- Separate the $...$ in many cases to get Latex to render properly.
      In output LaTeX, use incorrectly closed <span hidden> to pass in LaTeX options. -->
// ***I WILL MAKE THIS PSEUDOCODE INTO MY ALGORITHM.***

Function fastest_tour_nn returns time, orderedList of lighthouses.
1. **function** fastest_tour_nn(x, L):  <span hidden>\setlength\itemsep{0.0em}</div>
1. $~~~~$ *# MY ALGORITHM HERE, takes in foo, returns bar* <br>
   $~~~~$ *# acknowledging that this is example code and is not at all correct*
1. $~~~~$ newL $\gets$ ``list_minus``(L, x)   *# example of one of the functions*
1. $~~~~$ y $\gets$ L\[0\]   
1. $~~~~$ ntime, ntour $\gets$ ``fastest_tour``(y, newL)  *# example recursion*   
1. $~~~~$ tour $\gets$ \[x, y\]  *# example of creating an ordered list*
1. $~~~~$ time $\gets$ ``travel_time``(x, y)
1. $~~~~$ **return** time, tour


## 1b. English-language explanation (10 pts)

Write an English-language explanation of your pseudocode here

## 1c. Code implementation (25 pts)

Implement your pseudocode from above, using the following signature and code snippets.  Your code must have a reasonable, consistent, style and documentation. It must have appropriate data structures, modularity, and error checking.  Be sure to preserve the instructor input block, and do not change names of any of the variables - they will be inputted fresh by the instructor when testing your code.  Print the best tour and time, using the TRAVEL_TIME and L provided in the instructor input block, so that the correctness of your implementation can be validated.

In [2]:
####################################################
# INSTRUCTOR INPUT BLOCK
# THIS BLOCK WILL BE REPLACED BY INSTRUCTOR INPUTS
# DO NOT CHANGE THE NAMES OF THESE VARIABLES/METHODS
####################################################

TRAVEL_TIME = { 
      ('B', 'A') : 8.043412251828856 ,
      ('B', 'C') : 6.961562065036552 ,
      ('B', 'E') : 11.182761725279896 ,
      ('B', 'D') : 4.829491781522557 ,
      ('A', 'C') : 11.933637650024707 ,
      ('A', 'E') : 17.726993564286605 ,
      ('A', 'D') : 9.160385528861413 ,
      ('C', 'E') : 13.366783356602122 ,
      ('C', 'D') : 5.995980076893033 ,
      ('E', 'D') : 10.864682204416317 ,
}
# Additional test data is given at the bottom of the notebook.  You should also create your own test data as needed

# This function will populate a list L containing the names of the lighthouses
L = list(set([item for k in TRAVEL_TIME.keys() for item in k]))


In [3]:
# Utility functions that you can use if you wish

def list_minus(L, x):
    # Returns a list of L that does not have x in it
    return list(set(L)-set([x,]))

def travel_time(x, y):
    # Looks up x and y in TRAVEL_TIME in a way that order does not matter, returns a time
    global TRAVEL_TIME
    try:
        tm = TRAVEL_TIME[(x,y)] 
    except:
        tm = TRAVEL_TIME[(y,x)]
    return tm

def random_lighthouses(n):
    # Generates a random list of n lighthouses
    # returns a dictionary in the same format as TRAVEL_TIME and a list of lighthouses (new_L)
    
    from string import ascii_uppercase
    from random import uniform
    from itertools import combinations # students aren't allowed to use itertools for this assignment
    from math import sqrt
    
    new_TRAVEL_TIME = {}
    new_L = []
    letters = list(ascii_uppercase)
    
    for i in range(1,n):
        x = uniform(1, 10)
        y = uniform(1, 10)
        pt_name = letters[i-1]
        pt = (pt_name, (x,y))
        new_L.append(pt)
    
    pairs = list(combinations(new_L,2))
    for i in pairs:
        pt1 = i[0][1]
        pt2 = i[1][1]
        dist = sqrt((pt1[0]+pt2[0]**2 + (pt1[1]+pt2[1])**2))
        name = (i[0][0],i[1][0])
        new_TRAVEL_TIME[name] = dist
    return new_TRAVEL_TIME, new_L

def lighthouse_names(L):
    # Gets a list of the names of the lighthouses in dictionary L
    return list(set([item for k in TRAVEL_TIME.keys() for item in k]))
    
def mergeSort(inputList):
    # takes a list of values and returns a sorted list
    # if you use this, be sure to count the workload steps here
    #      consistent with the way you count them in your algorithm
    
    if len(inputList) > 1:
        mid = len(inputList) // 2
        left = inputList[:mid]
        right = inputList[mid:]

        # Recursive call on each half
        mergeSort(left)
        mergeSort(right)
        
        # Two iterators for traversing the two halves
        i = 0
        j = 0
        
        # Iterator for the main list
        k = 0
        
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
              # The value from the left half has been used
              inputList[k] = left[i]
              # Move the iterator forward
              i += 1
            else:
                inputList[k] = right[j]
                j += 1
            # Move to the next slot
            k += 1

        # For all the remaining values
        while i < len(left):
            inputList[k] = left[i]
            i += 1
            k += 1

        while j < len(right):
            inputList[k]=right[j]
            j += 1
            k += 1
            
    return (inputList)


In [ ]:
####################################################
# MY BRUTE FORCE FUNCTION
####################################################

# Put your code for fastests_tour_bf() from PA 2 or fastest_tour_nn() from PA 3 here
# to use as a utility function.  If your fastest_tour_memo() code is correct, it would be 
# better for you to use it than the brute force code as a benchmark... but it has to be
# correct or you will generate errors on your Monte Carlo experiment


In [8]:
####################################################
# MY NEAREST NEIGHBOR FUNCTION
####################################################

from math import inf

def fastest_tour_nn(start_light, L):
    # Accepts start_point (starting lighthouse name), list L (all lighthouses)
    # Returns best_tour (sequential list of lighthouses) and best_time (float value of best time in hours)
    # You must keep the signatures the same (accepts start_light, L and returns best_tour, best_time;
    #     start_light is a string, best_tour and L are lists of strings, and best_time is a float)
    # Your algorithm should implement the nearest neighbor approximation described above
    # Otherwise, you are free to change anything in here-- change variables, use a different structure,
    #     switch to object-oriented coding, etc.
    # Be sure that your pseudocode matches your actual code!

   
    best_tour = []  # used to store the running best overall tour that starts at start_light
    best_time = inf # used to store the time for the best_tour sequence
    
    # Nearest neighbor
    
    return best_tour, best_time  


In [9]:
####################################################
# KICKOFF CODE
# Suggested structure to kick off your calculations
# You will need to adjust this code to match your implementation
####################################################

from random import random.choice

start_light = random.choice(L)
L_minus = list_minus (start_light, L)
best_tour, best_time = fastest_tour_nn(start_light, L_minus)

print("The best tour is: ", ', '.join(best_tour))
print("The best time is: ", best_time)

The best tour is:  
The best time is:  inf


In [ ]:
####################################################
# CORRECTNESS CHECK
# Used by the grader to check correctness
# Chosen start_light to remove randomness
####################################################

TRAVEL_TIME = { 
      ('B', 'A') : 8.043412251828856 ,
      ('B', 'C') : 6.961562065036552 ,
      ('B', 'E') : 11.182761725279896 ,
      ('B', 'D') : 4.829491781522557 ,
      ('A', 'C') : 11.933637650024707 ,
      ('A', 'E') : 17.726993564286605 ,
      ('A', 'D') : 9.160385528861413 ,
      ('C', 'E') : 13.366783356602122 ,
      ('C', 'D') : 5.995980076893033 ,
      ('E', 'D') : 10.864682204416317 ,
}

L = list(set([item for k in TRAVEL_TIME.keys() for item in k]))

start_light = A
L_minus = list_minus (L, start_light)


# Uncomment one of these implementations
# If using a recursive step count:
best_tour, best_time = fastest_tour_nn(start_light, L_minus)

# If using a global step count:
#steps = 0
#best_tour, best_time, _ = fastest_tour_nn(start_light, L_minus)

print("The best tour is: ", ', '.join(best_tour))
print("The best time is: ", best_time)

# PART 2: ANALYSIS

## 2a. Asymptotic bounds (10 pts)

Using the techniques that you learned in Modules 1 and 2, provide an analytic estimate of the asymptotic bounds (Big-O, Big-Theta, etc.) for your algorithm. 


## 2b. Error bounds (10 pts)

What theoretical bound can you place on the best and worst case error that the nearest neighbor algorithm will have?

*Hint: the best-case scenario should be pretty obvious.  For the worst case, you will need to consider what geometric layout and choice of start point will cause the worst possible outcome.  Remember that you are working on a 2D Euclidean plane here, so the Triangle Inequality will likely come into play.*

## 2c. Monte Carlo experiment (25 pts)

Here, we will only need to look at the error (you don't have to graph the asymptotic bounds on the workload for this assignment).  In a Monte Carlo experiment, the model is run repeatedly on data sets that have random variations in the parameters.  In this case, what will change the outcome of the approximation algorithm will be the geometry of the lighthouses and the starting light.  So, the steps in the experiment will be:

1. Generate a new set of lighthouses, using the provided utility function
1. Run the brute force (or memoized) algorithm against the lighthouse list, to find the true best path
1. Run the nearest neighbor algorithm on the same lighthouse list
1. Compare the nearest neighbor best time to the true best time, to determine the error
1. Repeat this loop enough times to gather sufficient data to evaluate the goodness of the nearest neighbor approximation

*Hints: 
1. You will need to check this against varying numbers of lighthouses as well.  
1. Think carefully about what data would be most relevant to analyzing how "good" or "bad" the nearest neighbor approximation is-- what data should you collect?  Best/worst error?  Median error?  Mean error?  Standard deviation?
1. Consider also that the raw error will be larger if the total path length is longer-- should you normalize the value?  And how would you normalize it?
1. Allow plenty of time for the experiment-- the brute force algorithm is very slow, especially with larger n.  You will have to generate enough runs to ensure that you are getting a representative sample of errors-- more is better, as a rule, but you will not have time to run truly large-scale data runs here.  Begin with only small values for the number of iterations (say, five) and ensure that your code is working well before increasing the number of runs to a robust number for your final data run.

In [1]:
# MY CODE BLOCK TO EXPERIMENT WITH 3-10 LIGHTHOUSES AND GATHER ERROR DATA
# For your final data collection run, you should have at least 25 iterations for each number of lighthouses

### Plot
Plot your data here.  Use a graph type that gives meaningful information on the performance of the nearest neighbor algorithm-- pretend that you are briefing your boss on whether to switch a major business process from a brute force implementation to a nearest neighbor implementation-- what data would he/she find useful to quantify the tradeoff between the "optimum" calculation and the "good enough" calculation?

In [ ]:
####################################################
# PUT YOUR PLOT CODE HERE
# 
####################################################

# PART 3 RETROSPECTION

## Retrospection (10 pts)

Explain your results here.  Your graph should "tell a story"... write the story in plain English here.

If you had a problem with your code, or graph, write down here how you followed the 1-2-3 Rule given in the Getting Started module.  You should explain what you tried to do, what did/did not work, and what you would do next if you had additional time.  **Important:  don't just turn in broken code, or graphs that don't match your analytic results!**  It's important to show that, if there was some problem, you recognized that there was an issue and what you were doing to resolve the disconnect.  Bugs and mistakes do happen, time does run out, but you need to show what you understand of the problem in order to get at least partial credit.  

## Citations
Cite any help you received and any sources you referenced here

In [5]:
# INSTRUCTOR-PROVIDED TEST DATA
# Ensure that your code matches both the input signature and the expected output
# For grading, different test data will be pasted into the input cell and your code cells will be executed
# Be sure that your input and output signatures match the provided sample data
# Times are symmetrical, so the reverse order tour is also acceptable (e.g. A, B, C or C, B, A)

# Test #1
TRAVEL_TIME = { 
      ('D', 'E') : 9.8874546134365 ,
      ('D', 'B') : 8.650955785569098 ,
      ('D', 'C') : 4.527990409960845 ,
      ('D', 'A') : 9.817667809230786 ,
      ('E', 'B') : 10.931854306263975 ,
      ('E', 'C') : 7.255251488484818 ,
      ('E', 'A') : 12.917982527478712 ,
      ('B', 'C') : 4.113565483054365 ,
      ('B', 'A') : 9.560863383439097 ,
      ('C', 'A') : 7.854345573910511 ,
}
# Expected output
# The best tour is:  A, B, C, D, E
# The best time is:  28.089873889890804

# Test #2
TRAVEL_TIME = { 
      ('B', 'C') : 6.429795406216918 ,
      ('B', 'A') : 11.629846115160516 ,
      ('B', 'D') : 7.679251919404714 ,
      ('B', 'E') : 9.347706263090837 ,
      ('C', 'A') : 12.280646160363432 ,
      ('C', 'D') : 7.746192483295421 ,
      ('C', 'E') : 9.90681627370574 ,
      ('A', 'D') : 12.227183481562683 ,
      ('A', 'E') : 16.655823285647106 ,
      ('D', 'E') : 8.25715774835559 ,
}
# Expected output
# The best tour is:  A, B, C, D, E
# The best time is:  34.06299175302845

# Test #3
TRAVEL_TIME = { 
      ('F', 'E') : 7.453320453415392 ,
      ('F', 'D') : 6.170569410345761 ,
      ('F', 'I') : 10.448429302986911 ,
      ('F', 'G') : 6.187750187309644 ,
      ('F', 'C') : 12.090422838563583 ,
      ('F', 'H') : 11.539119418380032 ,
      ('F', 'A') : 13.23865323724485 ,
      ('F', 'J') : 14.209616157057711 ,
      ('F', 'B') : 12.029520235766265 ,
      ('E', 'D') : 4.594971038617467 ,
      ('E', 'I') : 9.488857351897519 ,
      ('E', 'G') : 4.661282508675182 ,
      ('E', 'C') : 10.705763401441896 ,
      ('E', 'H') : 10.12354365573923 ,
      ('E', 'A') : 12.05863087182219 ,
      ('E', 'J') : 12.857918364285274 ,
      ('E', 'B') : 10.915808926216425 ,
      ('D', 'I') : 8.773798408565863 ,
      ('D', 'G') : 3.549820998388679 ,
      ('D', 'C') : 9.084763991756446 ,
      ('D', 'H') : 8.47244200438249 ,
      ('D', 'A') : 10.768085646027655 ,
      ('D', 'J') : 11.205467989446557 ,
      ('D', 'B') : 9.811703475051996 ,
      ('I', 'G') : 4.856711290250502 ,
      ('I', 'C') : 10.303247633652786 ,
      ('I', 'H') : 9.72873923304563 ,
      ('I', 'A') : 11.752971702744057 ,
      ('I', 'J') : 12.386140947772116 ,
      ('I', 'B') : 10.715926552978804 ,
      ('G', 'C') : 8.939922836985131 ,
      ('G', 'H') : 8.325372714362043 ,
      ('G', 'A') : 10.658709470483634 ,
      ('G', 'J') : 11.05300320168352 ,
      ('G', 'B') : 9.726036954632448 ,
      ('C', 'H') : 14.85107596522508 ,
      ('C', 'A') : 16.127909792272288 ,
      ('C', 'J') : 17.54748278310382 ,
      ('C', 'B') : 14.699070399680458 ,
      ('H', 'A') : 15.723529687188293 ,
      ('H', 'J') : 17.10791004081554 ,
      ('H', 'B') : 14.306778662449995 ,
      ('A', 'J') : 16.949188359233272 ,
      ('A', 'B') : 14.239542023142393 ,
      ('J', 'B') : 16.6207970728817 ,
}
# Expected output
# The best tour is:  A, B, C, D, E, F, G, H, I, J
# The best time is:  86.69967098910159

# Test 4 - test against the actual lighthouses used in the challenge, so
# you can see the original motivation for this assignment.  Times are from
# maps.google.com and reflect best possible driving times between two 
# lighthouses subject to current traffic conditions.

# If you want to visualize this tour, it's here:
# https://goo.gl/maps/h9NbbQT5kS3S6kZ98

# Note that in the real world, travel times are highly dependent on time of
# day and are often not symmetrical-- that's part of why the TSP problem is
# so interesting!


TRAVEL_TIME = { 
    ('Concord Point', 'Seven Foot Knoll') : 0.88 ,
    ('Concord Point', 'Lightship Chesapeake') : 0.87 ,
    ('Concord Point', 'Hooper Strait') : 1.92 ,
    ('Concord Point', 'Choptank River') : 2.02 ,
    ('Concord Point', 'Drum Point') : 2.12 ,
    ('Concord Point', 'Cove Point') : 2.15 ,
    ('Concord Point', 'Piney Point') : 2.60 ,
    ('Concord Point', 'Point Lookout') : 2.73 ,
    ('Concord Point', 'Fort Washington') : 1.73 ,
    ('Concord Point', 'Sandy Point') : 1.28 ,
    ('Seven Foot Knoll', 'Lightship Chesapeake') : 0.07 ,
    ('Seven Foot Knoll', 'Hooper Strait') : 1.52 ,
    ('Seven Foot Knoll', 'Choptank River') : 1.62 ,
    ('Seven Foot Knoll', 'Drum Point') : 1.58 ,
    ('Seven Foot Knoll', 'Cove Point') : 1.62 ,
    ('Seven Foot Knoll', 'Piney Point') : 2.05 ,
    ('Seven Foot Knoll', 'Point Lookout') : 2.22 ,
    ('Seven Foot Knoll', 'Fort Washington') : 1.17 ,
    ('Seven Foot Knoll', 'Sandy Point') : 0.78 ,
    ('Lightship Chesapeake', 'Hooper Strait') : 1.47 ,
    ('Lightship Chesapeake', 'Choptank River') : 1.57 ,
    ('Lightship Chesapeake', 'Drum Point') : 1.53 ,
    ('Lightship Chesapeake', 'Cove Point') : 1.57 ,
    ('Lightship Chesapeake', 'Piney Point') : 1.98 ,
    ('Lightship Chesapeake', 'Point Lookout') : 2.17 ,
    ('Lightship Chesapeake', 'Fort Washington') : 1.12 ,
    ('Lightship Chesapeake', 'Sandy Point') : 0.73 ,
    ('Hooper Strait', 'Choptank River') : 0.60 ,
    ('Hooper Strait', 'Drum Point') : 2.03 ,
    ('Hooper Strait', 'Cove Point') : 2.08 ,
    ('Hooper Strait', 'Piney Point') : 2.50 ,
    ('Hooper Strait', 'Point Lookout') : 2.67 ,
    ('Hooper Strait', 'Fort Washington') : 1.77 ,
    ('Hooper Strait', 'Sandy Point') : 0.93 ,
    ('Choptank River', 'Drum Point') : 2.13 ,
    ('Choptank River', 'Cove Point') : 2.17 ,
    ('Choptank River', 'Piney Point') : 2.60 ,
    ('Choptank River', 'Point Lookout') : 2.77 ,
    ('Choptank River', 'Fort Washington') : 1.85 ,
    ('Choptank River', 'Sandy Point') : 1.03 ,
    ('Drum Point', 'Cove Point') : 0.23 ,
    ('Drum Point', 'Piney Point') : 0.48 ,
    ('Drum Point', 'Point Lookout') : 0.63 ,
    ('Drum Point', 'Fort Washington') : 1.18 ,
    ('Drum Point', 'Sandy Point') : 1.32 ,
    ('Cove Point', 'Piney Point') : 0.70 ,
    ('Cove Point', 'Point Lookout') : 0.83 ,
    ('Cove Point', 'Fort Washington') : 1.28 ,
    ('Cove Point', 'Sandy Point') : 1.35 ,
    ('Piney Point', 'Point Lookout') : 0.72 ,
    ('Piney Point', 'Fort Washington') : 1.42 ,
    ('Piney Point', 'Sandy Point') : 1.78 ,
    ('Point Lookout', 'Fort Washington') : 1.67 ,
    ('Point Lookout', 'Sandy Point') : 1.97 ,
    ('Fort Washington', 'Sandy Point') : 1.05 ,   
}

# Expected output
# The best tour is:  Choptank River, Hooper Strait, Sandy Point, Concord Point, Seven Foot Knoll, Lightship Chesapeake, 
#       Fort Washington, Cove Point, Drum Point, Piney Point, Point Lookout
# The best time is:  7.59